In [ ]:
# Leer ficheros de datos
import codecs
import pandas as pd
import pyarrow.parquet as pq
# Extraer información
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
# Auxiliares
import numpy as np
import logging
from typing import Union
import pyarrow as pa
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

### Cargamos el parquet

In [ ]:
df = pd.read_parquet('Datasets/PHA/PHA_Data.parquet', engine='pyarrow')


In [ ]:
df.dtypes

In [ ]:
df['c55']

### Convertimos cualquier tipo de registro faltante a NaN

In [ ]:
def clean_data(value): 
    # Si se trata de un valor en blanco sustituir a NaN 
    try: 
        if value in [None, 'nan']: 
            return np.nan 
         
        return value 
 
    except Exception as e: 
        return value 
     
# Iterar sobre todas las columnas y comprobar sus valores 
columns = len(df.columns) 
 
for column in df.columns: 
    # Utilizar compresion de listas y aplicar el filtrado de valores 
    values = [clean_data(value) for value in df[column]] 
    df[column] = values 
    columns -= 1 
    print(f'[+] REMAINING_COLUMNS\t{columns}\t', end='\r') 
     
df

### Cargamos cabecera  y convertimos a dataframe

In [ ]:
url_legend = 'https://av-info.faa.gov/data/AID/Afilelayout.txt'
# Leemos el contenido de la leyenda
with urlopen(url_legend) as content:
    soup = BeautifulSoup(content, "html.parser")
    soup_lines = str(soup).split('\r\n')

# Transformamos la respuesta en un diccionario con el nombre de la columna y la descripción
legend_df = {"Column_name": [], "Description": []}
# Recorrer las lineas con datos de lineas. Se salta las dos primeras y las tres ultimas
# filas por no tener datos relevantes de la leyenda
for line in soup_lines[2:-3]:
    # Se extrae los 5 primeros caracteres para conocer el nombre de la columna
    legend_df["Column_name"].append(line[0:5].strip())
    # Se extrae la descripcion, esta comienza en la posicion 53
    legend_df["Description"].append(line[53:].strip())
    
# Convertir el diccionario en pandas.Dataframe
legend_df = pd.DataFrame.from_dict(data=legend_df)
legend_df

### Enumeramos las cabecera con la descripción para poder verlas mejor y seleccionamos las columnas que podrían ser relevantes

In [ ]:
used_columns = ['c76','c1', 'c6', 'c7', 'c8', 'c10', 'c132', 'c134', 'c136', 'c138', 'c141', 'c160', 'c162', 'c23', 'c24', 'c11', 'c14', 'c20', 'c21', 'c102', 'c104', 'c106', 'c108', 'c110', 'c114', 'c115', 'c117', 'c118', 'c61', 'c62', 'c65', 'c67', 'c68', 'c41', 'c45', 'c47', 'c49', 'c50', 'c52', 'c53', 'c54', 'c55', 'c56', 'c120', 'c121', 'c122', 'c126', 'c127', 'c128', 'c130', 'c78', 'c80', 'c82', 'c84', 'c86', 'c88', 'c90', 'c92', 'c94', 'c96', 'c44', 'c46', 'c48', 'c51', 'c229', 'c230']
#used_columns = [f'c{idx}' for idx in used_columns]

for col in used_columns:
    description = legend_df[legend_df["Column_name"] == col]["Description"].values[0]
    print(f'[+]  Descripción columnas\t{col}\t{description}')
   

### Creamos dataframe a partir del primario usando solamente las columnas de la lista anteriormente  generada

In [ ]:
data= df.loc[:, used_columns]

In [ ]:
repeated_values = data['c76'].value_counts()
repeated_values_df = repeated_values.to_frame()
repeated_values_df.columns = ['Frequency']
print(repeated_values_df)

### Realizamos un conteo de los datos faltantes NaN de las columnas que pueden ser potenciales

In [ ]:
# Contar el número de valores NaN en cada columna
print(data.isna().sum())

### Procedemos a borrar las columnas con demasiados datos faltantes (más de 100.000 unidades) y las filas que tengan más de 3 valores faltantes

In [ ]:
#Borramos columnas que tengan más de 100.000 datos faltantes  
data = data.dropna(thresh=100000, axis=1)
data = data.dropna(thresh=3)

### Volvemos a contar despues del borrado para ver que columnas son aprovechables

In [ ]:
# Contar el número de valores NaN en cada columna
print(data.isna().sum())



In [ ]:
columnas = list(data.columns)
#unused_columns = [5, 3, 2, 4, 9, 75, 140, 139, 203, 204, 214, 790, 26, 37, 15, 16, 17,18, 19, 143, 205, 206, 207, 208, 210, 43, 129, 124, 125, 77]
#unused_columns = [f'c{idx}' for idx in unused_columns]

for col in columnas:
    description = legend_df[legend_df["Column_name"] == col]["Description"].values[0]
    print(f'[+] Descripción columnas\t{col}\t{description}')

# ESTUDIO DE COLUMNAS PARA SABER CUALES SON CATEGORICAS Y CUALES NUMERICAS PARA PROCEDER SEGUIDAMENTE A LA IMPUTACIÓN

Si se optara por hacerlo de forma automatica hay atributos que se clasificaran como categoricos cuando no lo son y viceversa
Por lo que hay que proceder a hacerlo manualmente mediante dos pasos:
En el primer paso se comprueba la cantidad de valores unicos clasificando los que superan los 100 valores como numericos. 
En segundo lugar hay que comprobar que los valores no unicos no contengan string


### Primer paso

In [ ]:
# con  un sencillo for podemos apreciar la cantidad de valores unicos de cada columna
# vamos a establecer que los que tengan más de 100 valores unicos son numericos y los restantes categoricos

for col in data.columns:
    print(col, data[col].nunique())

In [ ]:
#Vamos a establecer como criterio de clasificación las que superen 100 caracteres unicos serán numericas
#las restantes serán categoricas

# Definimos una lista para almacenar las columnas numéricas
num_cols = []

# Definimos una lista para almacenar las columnas categóricas
cat_cols = []

# Verificamos la cardinalidad de cada columna
for col in data.columns:
    unique_vals = data[col].nunique()
    if unique_vals >= 100:
        num_cols.append(col)
    else:
        cat_cols.append(col)

# Imprimimos las columnas numéricas y categóricas
print("Columnas numéricas:", num_cols)
print("Columnas categóricas:", cat_cols)

In [ ]:
for col in num_cols:
    try:
        numeric_values = pd.to_numeric(data[col])
        non_numeric_values = numeric_values.isna().sum()
        print(f"La columna '{col}' tiene {non_numeric_values} valores no numéricos (excluyendo NaN)")
    except ValueError:
        print(f"La columna '{col}' contiene valores no numéricos")

### Segundo paso

In [ ]:
for i in num_cols:
    print(data[i])

In [ ]:
for i in cat_cols:
    print(data[i])

Hay que añadir las columnas con  texto es decir que no son tipo numerico y pasarlas a categoricas.

In [ ]:
cat_cols=['c76','c1', 'c6', 'c7', 'c8', 'c132', 'c134', 'c141', 'c11', 'c102', 'c104', 'c106', 'c108', 'c110', 'c65', 'c67', 'c68', 'c41', 'c45', 'c47', 'c49', 'c50', 'c52', 'c130', 'c80', 'c82', 'c94', 'c96', 'c44', 'c46', 'c48', 'c51','c23','c24','c14','c78']
num_cols= ['c10', 'c61', 'c62', 'c53', 'c54', 'c55', 'c56']

In [ ]:
data

In [ ]:

# Aplicar la conversión en la columna 'A' usando apply y una función lambda
data['c76'] = data['c76'].apply(lambda x: 0 if x <= 3 else 1)

print(data)

In [ ]:
data.dtypes

In [ ]:
data["c10"] = data["c10"].apply(pd.to_numeric, downcast='float', errors='coerce')

In [ ]:
data['c10']

In [ ]:
data.dtypes

In [ ]:
repeated_values = data['c76'].value_counts()
repeated_values_df = repeated_values.to_frame()
repeated_values_df.columns = ['Frequency']
print(repeated_values_df)

In [ ]:
data

# Imputación

## Imputación simple y menos precisa

## Imputación más exacta pero más costosa computacionalmente

#### Imputación de columnas  numericas

In [ ]:
import pandas as pd
from sklearn.impute import KNNImputer, IterativeImputer

# Imputación para columnas numéricas
#num_cols = data.select_dtypes(include=['float', 'int']).columns.tolist()
num_imputer = KNNImputer(n_neighbors=5)
data[num_cols] = num_imputer.fit_transform(data[num_cols])



In [ ]:
data[num_cols]

In [ ]:
data.dtypes

In [ ]:
data[cat_cols]

#### Imputación de columnas categoricas

In [ ]:
data["c76"]

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

df_imputed = data
# Imputación de valores categóricos
#cat_cols = data.select_dtypes(include='object').columns.tolist()
for col in cat_cols:
    le = LabelEncoder()
    df_imputed[col] = le.fit_transform(df_imputed[col].astype(str))

imputer = KNNImputer()
df_imputed[cat_cols] = imputer.fit_transform(df_imputed[cat_cols])

# Convertir de vuelta a valores categóricos
for col in cat_cols:
    le = LabelEncoder()
    le.fit(df_imputed[col].astype(int))
    df_imputed[col] = le.inverse_transform(df_imputed[col].astype(int))

In [ ]:
df_imputed.dtypes

In [ ]:
data["c76"]

In [ ]:
data

In [ ]:
print(data.isna().sum())
print(data.shape)

In [ ]:
data.dtypes

In [ ]:
for col in data.columns:
    print(col, data[col].nunique())

# Pasamos a parquet con los datos ya imputados para no tener que imputarlo cada vez que utilicemos el código

In [ ]:
data.to_parquet("data_tratamiento.parquet")

AL LEER EL PARQUET TODOS LOS ATRIBUTOS APARECEN COMO NUMERICOS O FLOAT Y NINGUNO CATEGORICO POR LO QUE TENDREMOS QUE VOLVER A INDICAR CUALES SON CATEGORICOS Y CULES NO

### Ver relación entre caracteristicas (uso de SKEW())para ver como se relacionan los atributos y poder ver también si presentan valores atipicos

In [ ]:
# Leer ficheros de datos
import codecs
import pandas as pd
import pyarrow.parquet as pq
# Extraer información
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
# Auxiliares
import numpy as np
import logging
from typing import Union
import pyarrow as pa
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression




#Leemos parquet y generamos dataframe para seguir trabajando con el
data = pd.read_parquet('data_tratamiento.parquet', engine='pyarrow')

In [ ]:
data.skew()

Podemos verque una de los atributos concretamente el c55 es posible que tenga un valor atípico dada la desviacion que tiene

Seguidamente hacemos un boxplot para poder confirmar si presenta valores atipicos 

In [ ]:
# Crear un boxplot de una sola columna del dataframe
fig, ax = plt.subplots(figsize=(8,6))
ax.boxplot(data['c55'])

# Personalizar el gráfico
ax.set_title('Boxplot de la columna')
ax.set_ylabel('Valores')
plt.ylim(0, 4000)
# Mostrar el gráfico
plt.show()

Efectivamente es muy posible, contaremos ahora la cantidad de filas en las que se repiten estos valores para poder ver si es algo frecuente o es algo puntual que tiene bastante probabilidad de ser valor erroneo

In [ ]:
# Supongamos que la columna se llama 'nombre_columna' y está en el dataframe 'df'
conteo = data['c55'].between(1200, 4000, inclusive=True).sum()

print(f"El número de filas con valor entre 2000 y 4000 es: {conteo} de {len(data)} filas totales del dataframe")

Procederemos a eliminar esos tres registros  que tomaremos como erroneos

In [ ]:
columna = data['c55']
mascara = columna.between(1200, 4000, inclusive=True)
#mascara.count() 
data = data[~mascara].dropna()

conteo = data['c55'].between(1200, 4000, inclusive=True).sum()
print(f"El número de filas con valor entre 2000 y 4000 es: {conteo} de {len(data)} filas totales del dataframe")

Tras el tratamiento a la posibilidad de valores erraticos debido a la desviación que presenta la columna c55 observamos que los datos son aparentemente correctos y es la naturaleza de los mismos la que produce esa desviación.C6 aparenta tener el mismo problema pero es también debido a que es una fecha.

In [ ]:
data.skew()

# Escalar los datos

In [ ]:
from sklearn.preprocessing import StandardScaler

cat_cols=['c76','c1', 'c6', 'c7', 'c8', 'c132', 'c134', 'c141', 'c11', 'c102', 'c104', 'c106', 'c108', 'c110', 'c65', 'c67', 'c68', 'c41', 'c45', 'c47', 'c49', 'c50', 'c52', 'c130', 'c80', 'c82', 'c94', 'c96', 'c44', 'c46', 'c48', 'c51','c23','c24','c14','c78']
num_cols= ['c10', 'c61', 'c62', 'c53', 'c54', 'c55', 'c56']

df = pd.DataFrame(data)
print(df.dtypes)
# Seleccionamos los atributos numéricos y los estandarizamos
#num_cols = df.select_dtypes(include=['int', 'float']).columns
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

# Creamos un nuevo dataframe con los datos categóricos y los datos numéricos estandarizados
#cat_cols = df.select_dtypes(include=['object']).columns
df = pd.concat([df[cat_cols], df[num_cols]], axis=1)

print(df)

Una vez ya realizado el tratamiento de los datos guardamos en un .parquet para proseguir con la predicción

In [ ]:
repeated_values = df['c76'].value_counts()
repeated_values_df = repeated_values.to_frame()
repeated_values_df.columns = ['Frequency']
print(repeated_values_df)

In [ ]:
df.to_parquet("data_tratamiento_escalado.parquet")

# Balancear el dataframe porque hay menos valores de accidente que de no accidentes en la variable objetivo c76

In [1]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
import pandas as pd

# Cargar los datos en un dataframe
df = pd.read_parquet('data_tratamiento_escalado.parquet', engine='pyarrow')

# Dividir los datos en conjuntos de entrenamiento y prueba
X = df.drop('c76', axis=1) # todas las columnas excepto la etiqueta
y = df['c76']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Sobremuestrear la clase minoritaria en el conjunto de entrenamiento
oversampler = RandomOverSampler(random_state=42)
X_train, y_train = oversampler.fit_resample(X_train, y_train)



In [8]:
y_train

0         0
1         0
2         0
3         0
4         0
         ..
298937    1
298938    1
298939    1
298940    1
298941    1
Name: c76, Length: 298942, dtype: int32

In [10]:
repeated_values = y_train.value_counts()
repeated_values_df = repeated_values.to_frame()
repeated_values_df.columns = ['Frequency']
print(repeated_values_df)

   Frequency
0     149471
1     149471


# Predecir

Predicción ejecutada en keras por problemas de ejecución en equipo local

https://www.kaggle.com/josepy/prediccion-final/edit

In [11]:
# Importar las bibliotecas necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


# Entrenar el modelo de Bosque Aleatorio
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Hacer predicciones sobre el conjunto de prueba
y_pred = rf.predict(X_test)

# Evaluar la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo: {:.2f}%".format(accuracy * 100))

Precisión del modelo: 99.58%
